# Baseline Popular Recommender
As with any ML project, we start with a non-ML approach as our baseline. It's very common in RecSys project to have popular recommender as not only a benchmark but also an actual component (retrieval stage).

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "001-baseline-popular"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-26 22:58:51.747 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 001-baseline-popular...
2024/09/26 22:58:51 INFO mlflow.tracking.fluent: Experiment with name 'FSDS RecSys - L4 - Reco Algo' does not exist. Creating a new experiment.


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "001-baseline-popular",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/001-baseline-popular",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")

# Implement

In [5]:
popular_items_df = (
    train_df.groupby(args.item_col, as_index=False)
    .size()
    .assign(
        score=lambda df: df["size"] / df["size"].max(),
        rec_ranking=lambda df: df["score"]
        .rank(method="first", ascending=False)
        .astype(int),
    )
    .sort_values(["rec_ranking"], ascending=[True])
)

top_popular_items_df = popular_items_df.head(args.top_K)

top_popular_items_df

,parent_asin,size,score,rec_ranking
2143,B0086VPUHI,1502,1.000000,1
3554,B01N3ASPNV,1502,1.000000,2
4337,B07YBXFDYN,1415,0.942077,3
2368,B00BGA9WK2,1253,0.834221,4
1775,B004RMK5QG,1109,0.738349,5
...,...,...,...,...
1719,B004I1JTEK,341,0.227031,96
4681,B0C5K4M7WJ,339,0.225699,97
2776,B00KWOSQC8,337,0.224368,98
3741,B072V478NR,337,0.224368,99


In [6]:
recommendations_df = (
    val_df[[args.user_col]]
    .drop_duplicates()
    .assign(tmp=1)
    .pipe(
        lambda df: pd.merge(
            df, top_popular_items_df.assign(tmp=1), on="tmp", how="left"
        )
    )[[args.user_col, args.item_col, "score", "rec_ranking"]]
)

recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B0086VPUHI,1.000000,1
1,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B01N3ASPNV,1.000000,2
2,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B07YBXFDYN,0.942077,3
3,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B00BGA9WK2,0.834221,4
4,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B004RMK5QG,0.738349,5
...,...,...,...,...
70795,AE4VXAYAK7QZ2PXHJNZK4UR5P63A,B004I1JTEK,0.227031,96
70796,AE4VXAYAK7QZ2PXHJNZK4UR5P63A,B0C5K4M7WJ,0.225699,97
70797,AE4VXAYAK7QZ2PXHJNZK4UR5P63A,B00KWOSQC8,0.224368,98
70798,AE4VXAYAK7QZ2PXHJNZK4UR5P63A,B072V478NR,0.224368,99


# Evaluate

## Ranking metrics

In [7]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
509,AFUQQLR2N2LY7XPE4VJ5YF3LDZVA,B07YN82X3B,5.0,1.0
250,AGJO7OFBOKRLDTSEL2HHSZSKDQ4Q,B07PZ8NZSZ,1.0,1.0
1549,AF4QBZD2EXOTKIOOH4BOC4HZDHYA,B08NRVRF3J,3.0,1.0
395,AH6YPZLRQH6OSGLGBVTGCNSF7JQQ,B0BVVTQ5JP,5.0,1.0
1325,AESEOKCWWKUG7YPP43J2CRWAXQIA,B09GM4283G,5.0,1.0
...,...,...,...,...
1365,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00LMRL00O,0.0,18.0
528,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00Z9TM72Q,0.0,18.0
1670,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07NKN4VR4,0.0,18.0
311,AFB6FYPPCN33UMUU5536IHXNOHCQ,B002Z01QO2,0.0,19.0


In [8]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
26,AE2AZ2MNROPF33U6SS53VI22OXJA,B0086VPUHI,1.000000,1,0,NaN
79,AE2AZ2MNROPF33U6SS53VI22OXJA,B01N3ASPNV,1.000000,2,0,NaN
90,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YBXFDYN,0.942077,3,0,NaN
32,AE2AZ2MNROPF33U6SS53VI22OXJA,B00BGA9WK2,0.834221,4,0,NaN
19,AE2AZ2MNROPF33U6SS53VI22OXJA,B004RMK5QG,0.738349,5,0,NaN
...,...,...,...,...,...,...
72409,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00KWOSQC8,0.224368,98,0,NaN
72430,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B072V478NR,0.224368,99,0,NaN
72400,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00HGLLRV2,0.223036,100,0,NaN
72349,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00005AV8O,NaN,101,0,2.0


In [9]:
ranking_report = log_ranking_metrics(args, eval_df)

## Classification metrics

In [10]:
eval_classification_df = pd.merge(
    val_df,
    popular_items_df[[args.item_col, "score"]],
    on=[args.item_col],
    how="left",
    validate="m:1",
).assign(label=lambda df: df[args.rating_col].gt(0).astype(int))
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,score,label
0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B00V5V3E38,0.0,1633967679730,10525,4119,Video Games,Legend of Kay Anniversary - Nintendo Wii U,"[10 years after its initial release, Legend of...","[Video Games, Legacy Systems, Nintendo Systems...",None,"[-1, -1, -1, -1, 3766, 4102, 2872, 2810, 502, ...",0.025965,0
1,AGFPKXT34G5FGWARKXZC4GJTJQUQ,B01GY3651O,0.0,1630491281842,11724,2977,Video Games,XCOM 2 Deluxe Edition [Online Game Code],[The XCOM 2 Digital Deluxe Edition includes th...,"[Video Games, PC, Games]",None,"[594, 4578, 3289, 4664, 1057, 3845, 3010, 2691...",0.037284,0
2,AH4AOFTTDPHPAFAAVFMAF25H2LIQ,B09B14PJCG,0.0,1641748747823,7401,4682,Video Games,A Plague Tale: Innocence (XB1) - Xbox One,[Follow the grim tale of young Amicia and her ...,"[Video Games, Xbox One, Games]",59.99,"[4064, 24, 1441, 1805, 2619, 2254, 2274, 1454,...",0.041278,0
3,AEXTTZIJDNXIXQZFR5O7IJRXO3GA,B081243BT6,0.0,1637074638494,5074,3104,Cell Phones & Accessories,Orzly Carrying case for Nintendo Switch OLED a...,[],"[Video Games, Nintendo Switch, Accessories, Ca...",29.99,"[-1, -1, -1, -1, -1, 811, 2540, 313, 4394, 1846]",0.209055,0
4,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,1616,4127,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[1459, 1860, 3264, 569, 2143, 773, 4483, 296, ...",0.123835,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AFPSHZKKUL2YDGIDBQQUNRGE5MXQ,B07DD7QTBM,0.0,1641121394999,1672,2700,None,Just Dance 2019 - Xbox One Standard Edition,"[Dance to your own beat with Just Dance 2019, ...","[Video Games, Xbox One, Games]",12.3,"[-1, -1, -1, 2830, 1342, 1294, 1749, 2558, 385...",0.017310,0
1894,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,49,1887,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 3179, 1489, 2225, 3399, 3142, 4247, 3801,...",0.069241,1
1895,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,6619,2246,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 2780, 3158, 130, 1164, 1030]",0.093875,1
1896,AEBTSECUK7ZEECNSRHQLMKO3E5VA,B002BSA388,0.0,1642567970979,18651,3724,Video Games,Super Mario Galaxy 2,"[Product Description, Launch into a new univer...","[Video Games, Legacy Systems, Nintendo Systems...",80.82,"[-1, -1, -1, -1, 2135, 2729, 14, 3639, 3118, 955]",0.136485,0


In [11]:
eval_classification_df["score"].describe().T

count    1898.000000
mean        0.113113
std         0.157735
min         0.008655
25%         0.027963
50%         0.053262
75%         0.123169
max         1.000000
Name: score, dtype: float64

In [12]:
classification_report = log_classification_metrics(
    args, eval_classification_df, target_col="label", prediction_col="score"
)

## Loss
We can estimate what kinds of MSE loss should be the upper bound model training based on measuring the loss when asking the model to naively predict the mean rating for every item.

In [13]:
naive_prediction = train_df[args.rating_col].mean()

naive_mse = (
    (
        val_df[args.rating_col]
        - val_df.assign(naive_prediction=naive_prediction)["naive_prediction"]
    )
    .apply(lambda s: s * s)
    .mean()
)
logger.info(
    f"Val MSE = {naive_mse:,.2f} given naive_prediction={naive_prediction:,.1f}"
)

2024-09-26 22:58:54.180 | INFO     | __main__:<module>:11 - Val MSE = 5.12 given naive_prediction=2.2


# Clean up

In [14]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/26 22:58:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run 001-baseline-popular at: http://localhost:5003/#/experiments/3/runs/a2221915dfce4e53b9f720352fa50521.
2024/09/26 22:58:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/3.
